In [16]:
import itertools
import operator
import sys
import os
sys.path.append("../src")
from glob import glob


In [17]:
search_root = "/checkpoint/jth/neuraldb/final=true"
checkpoint_name = "best_em.ckpt"
files = glob("{}*/**/{}".format(search_root,checkpoint_name), recursive=True)


print(len(files))

144


In [3]:
def expand(idx,chunk):
  #if idx == 0:
  #  return ["job_id={}".format(chunk)]
  #elif idx == 1:
  #  return ["experiment={}".format(chunk)]
  if chunk.startswith("seed-"):
    return ["seed={}".format(chunk.replace("seed-",""))]
  elif "," in chunk:
    return chunk.split(",")
  elif "=" in chunk:
    return [chunk]
  return []

experiments = []
for file in files:
    chunks = file.replace(search_root,"").split("/")
    chunks = itertools.chain(*[expand(idx, chunk) for idx, chunk in enumerate(chunks)])

    data = {k:v for k,v in (chunk.split("=") for chunk in chunks)}
    data["file"] = file
    data['dir'] = os.path.dirname(file)
    experiments.append(data)




In [15]:
for exp in experiments:
    version = exp["version"]


    if "dpr" not in exp["file"]:
        continue

    retriever = "all" if exp["experiment"] == "entire_database" else ("pipeline" if "pipeline" in exp["experiment"] else "tfidf" )
    module = "neuraldb.commands.t5train2" if "fusion" not in exp["file"] else "neuraldb.commands.t5train2_fusion"
    iterate = "--iterate" if exp["experiment"] == "iterate_database" else ""
    oracle = "--oracle" if "oracle" in exp["experiment"] else ""

    print("python -m {} "
          "--output_dir {} "
          "--model_name_or_path t5-base "
          "--do_predict "
          "--train_batch_size 16 "
          "--eval_batch_size 16 "
          "--n_gpu 4 "
          "--train_path={}/train_queries_last_{}.json "
          "--val_path={}/dev_queries_last_{}.json "
          "--test_path={}/dev_queries_last_{}.json "
          "--retriever {} "
          "{} {} "
          "--dataset_version={}".format(module, exp['dir'],version,exp['context'],version,exp['context'],version,exp['context'],retriever, iterate, oracle, version))



python -m neuraldb.commands.t5train2 --output_dir /checkpoint/jth/neuraldb/final=true/experiment=pipeline/dataset=ndb,model=t5-base,version=v0.5,pipeline=dpr/lr=5e-4/context=50,filters=null,max_queries=null/seed-1 --model_name_or_path t5-base --do_predict --train_batch_size 16 --eval_batch_size 16 --n_gpu 4 --train_path=v0.5/train_queries_last_50.json --val_path=v0.5/dev_queries_last_50.json --test_path=v0.5/dev_queries_last_50.json --retriever pipeline   --dataset_version=v0.5
python -m neuraldb.commands.t5train2 --output_dir /checkpoint/jth/neuraldb/final=true/experiment=pipeline/dataset=ndb,model=t5-base,version=v0.5,pipeline=dpr/lr=5e-4/context=50,filters=null,max_queries=null/seed-3 --model_name_or_path t5-base --do_predict --train_batch_size 16 --eval_batch_size 16 --n_gpu 4 --train_path=v0.5/train_queries_last_50.json --val_path=v0.5/dev_queries_last_50.json --test_path=v0.5/dev_queries_last_50.json --retriever pipeline   --dataset_version=v0.5
python -m neuraldb.commands.t5trai